In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [11]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [5]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

### Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [12]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [14]:
W = eval_model(X_st, y, iterations=6000, eta=1e-3)

0 [ 0.49633477 -0.13971518  0.64766116  1.52246371] 1.1785958344356262
600 [ 0.34705426 -0.6447563   0.63955258  1.31689489] 0.5381879835818275
1200 [ 0.28204505 -0.70954957  0.67640202  1.28343064] 0.5169108706925747
1800 [ 0.23338762 -0.71687553  0.71528278  1.27964103] 0.5102540676032226
2400 [ 0.19059208 -0.71650506  0.75024874  1.28315668] 0.5051334973384451
3000 [ 0.15151323 -0.71596113  0.78093944  1.28996596] 0.500934768374212
3600 [ 0.11541447 -0.71633584  0.80776306  1.2990673 ] 0.49742112841177233
4200 [ 0.08184421 -0.71763719  0.8312054   1.31001282] 0.4944212759367386
4800 [ 0.05043893 -0.71971682  0.85171175  1.32247987] 0.4918080086821225
5400 [ 0.02088732 -0.72242503  0.86967126  1.33619746] 0.48948736847738183


### Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [15]:
def calc_pred_proba(W, X):
    pred = 1 / (1 + np.exp(-1 * np.dot(X, W)))
    return pred

In [16]:
calc_pred_proba(W, X_st)

array([0.43805063, 0.52899538, 0.69692326, 0.04884659, 0.79982868,
       0.73581582, 0.79837588, 0.20767653, 0.4157185 , 0.78400761])

### Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [17]:
def calc_pred(W, X):
    y_pred = np.array([1 if el >= 0.5 else 0 for el in calc_pred_proba(W, X)])
    return y_pred

In [18]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

### Могла ли модель переобучиться? Почему?

Да. Из-за большого числа итераций и подбора eta, а также отсутствия кросс-валидации, модель могла переобучиться и просто запомнить результаты.